pymysql to connect to mysql instance running on RDS
pandas to be able to import the dataset as a frame

In [0]:
pip install pymysql

     |████████████████████████████████| 51kB 3.4MB/s 


In [0]:

import pymysql
import pandas as pd

**Connecting to RDS Mysql to read it into dataframe df**

In [0]:
connection = pymysql.connect(host='database-1.crfkl2juooug.ap-southeast-1.rds.amazonaws.com',
                             user='prithvi',
                             password='NannaMooda',                             
                             db='micfinml',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)


        
df = pd.read_sql('SELECT * FROM micfinml.creditDefault_nonbias', con=connection)
 
 
connection.close()

In [0]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,RS_0,RS_2,RS_3,RS_4,RS_5,RS_6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,p_default
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,0,1000,1000,1000,0,2000,1
2,14,70000,1,2,2,30,1,2,2,0,0,2,3200,0,3000,3000,1500,0,1
3,17,20000,1,1,2,24,0,0,2,2,2,2,3200,0,1500,0,1650,0,1
4,22,120000,2,2,1,39,-1,-1,-1,-1,-1,-1,316,316,0,632,316,0,1


In [0]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'RS_0',
       'RS_2', 'RS_3', 'RS_4', 'RS_5', 'RS_6', 'PAY_AMT1', 'PAY_AMT2',
       'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'p_default'],
      dtype='object')

**Check for bias**

In [0]:
df['p_default'].value_counts()

0    6636
1    6636
Name: p_default, dtype: int64

Choose the significant columns. Drop ID and call the new dataframe df_without_id

In [0]:
df_without_id = df.drop('ID', axis=1)

In [0]:
df_without_id.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'RS_0', 'RS_2',
       'RS_3', 'RS_4', 'RS_5', 'RS_6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'p_default'],
      dtype='object')

**Now, split our DataFrame into input data and target**

In [0]:
data = df_without_id[['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'RS_0', 'RS_2',
       'RS_3', 'RS_4', 'RS_5', 'RS_6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']]

In [0]:
data.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'RS_0', 'RS_2',
       'RS_3', 'RS_4', 'RS_5', 'RS_6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'],
      dtype='object')

In [0]:
target = df_without_id[['p_default']]

In [0]:
target.columns

Index(['p_default'], dtype='object')

In [0]:
target.head()

,p_default
0,1
1,1
2,1
3,1
4,1


**Defining the Train and Test dataset**
--- train_test_split function splits random train and test subsets

In [0]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data, target,
                                                    random_state=42)

**Train a Decision tree model**

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler


dtc = DecisionTreeClassifier(random_state=42, max_leaf_nodes=10, max_depth=5)
dtc.fit(train_data, train_target)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [0]:
dtc.predict(test_data)

array(['1', '0', '1', ..., '1', '0', '1'], dtype=object)

**Run the Test on a the test set**

In [0]:
pred = dtc.predict(test_data)
print(pred)

['1' '0' '1' ... '1' '0' '1']


**Convert the Test results into a DataFrame**

In [0]:
test_data_frame = pd.DataFrame(test_data)
predicted_test_data_frame = pd.DataFrame(pred)

Run the Test with a new data set from an external source.
The external dataset comes as an Array
Use Numpy to reshame the Array into a 2D array as the model takes an input as a 2D Array

In [0]:
import numpy as np



In [0]:
x1 = [1,2000,2,2,1,24,2,2,-1,-1,-2,-2,0,689,0,0,0]
A = np.array(x1)
B = np.reshape(A,(1,17))
#pd.DataFrame(x1)

In [0]:
print (B)

[[   1 2000    2    2    1   24    2    2   -1   -1   -2   -2    0  689
     0    0    0]]


Run the model using the 2-D Array. And note the predicted value

In [0]:
pred1 = dtc.predict(B)
print(pred1)

['1']
